# doubledot.Salesforce
> Salesforce class for transfering data from Vantix to Salesforce

In [ ]:
#| default_exp crema_sf

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| exporti 
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os
from doubledot import ATMS_api

In [ ]:
#| export
## Module for Salesforce API

class Salesforce:
    """Class for Salesforce API"""
    class_download_dir = os.path.join(os.getcwd(),'sf_download')

    def __init__(self):
        # set up access token 
        self._sf_access_token = self.get_token_with_REST()
        self.bulk_job_id = None
        self.atms = None

        # create unique download directory per instance
        if not os.path.exists(Salesforce.class_download_dir):
            os.makedirs(Salesforce.class_download_dir)
            print(f"Directory 'atms_download' created successfully.")
        else:
            print(f"Directory 'atms_download' already exists.")

    def get_token_with_REST(self ):
        """retieve the access token from Salesforce

        Returns:
            string: the access token 
        """
        with open('secrets.json') as f:
            secrets = json.load(f)
        
        DOMAIN = secrets['instance']
        payload = {
            'grant_type': 'password',
            'client_id': secrets['client_id'],
            'client_secret': secrets['client_secret'],
            'username': secrets['username'],
            'password': secrets['password'] + secrets['security_token']
        }
        oauth_url = f'{DOMAIN}/services/oauth2/token'

        auth_response = requests.post(oauth_url, data=payload)
        return auth_response.json().get('access_token') ######## <<<<<<<<<<<<<<<< .       


    @property
    def sf_access_token(
        self 
     ) -> str : #the access toke
        """a @property
        retrieve token for Salesforce - verifies that token is still valid and attempts to get a new one from Salesforce site if not
        """
        if not(self.test_token()):
            self._sf_access_token = self.get_token_with_REST()
            # check to see if getting token worked
            assert (self.sf_access_token), "Fetching new token didn't fix problem"
        return self._sf_access_token
    
    @staticmethod
    def list_files():
        return os.listdir(Salesforce.class_download_dir)

show_doc(Salesforce.sf_access_token)
   

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/crema_sf.py#L63){target="_blank" style="float:right; font-size:smaller"}

### Salesforce.sf_access_token

>      Salesforce.sf_access_token ()

a @property
retrieve token for Salesforce - verifies that token is still valid and attempts to get a new one from Salesforce site if not

In [ ]:
#| export
@patch
def get_token_with_REST(self: Salesforce):
    """retieve the access token from Salesforce

    Returns:
        string: the access token 
    """
    with open('secrets.json') as f:
        secrets = json.load(f)
    
    DOMAIN = secrets['instance']
    payload = {
        'grant_type': 'password',
        'client_id': secrets['client_id'],
        'client_secret': secrets['client_secret'],
        'username': secrets['username'],
        'password': secrets['password'] + secrets['security_token']
    }
    oauth_url = f'{DOMAIN}/services/oauth2/token'

    auth_response = requests.post(oauth_url, data=payload)
    return auth_response.json().get('access_token') ######## <<<<<<<<<<<<<<<< .       



In [ ]:
#| export
@patch
def test_token(self: Salesforce):
    """Verify that token is still valid. If it isn't, it attempts to get a new one.

    Returns:
        boolean: true if token is valid, false otherwise
    """
    sf_headers = { 'Authorization': f"Bearer {self._sf_access_token}", 'Content-Type': 'application/json' }
    end_point ="https://cremaconsulting-dev-ed.develop.my.salesforce.com"
    service = "/services/data/v57.0/"
    r = requests.request("GET", end_point+service+f"limits", headers=sf_headers, data={})
    valid_token = r.status_code == 200
    if not(valid_token): print(r.status_code, type(r.status_code))
    return valid_token
    


In [ ]:
#| export
@patch
def create_job(self: Salesforce, 
                sf_object: str ='Contact', # the Salesforce object were going to operate on. 
                operation: str ='insert', # the database operation to use. Can be "insert","upsert" or "delete"
                external_id: str = 'External_Id__c' # when using "upsert", this field is used to identify the record
                )-> requests.Response :
    """Get job_id from Salesforce Bulk API

    """
    # Args: 
    #     sf_object (str, optional): the Salesforce object were going to operate on. Defaults to 'Contact'.
    #     operation (str, optional): ∆. Defaults to 'insert'.
    #     external_id (str, optional): the external id field for upsert operations. Defaults to 'External_Id__c'.
    #     sf_object (str, optional): the Salesforce object were going to operate on. Defaults to 'Contact'.
    #     operation (str, optional): the operation that will be used against the object. Defaults to 'insert'.
    #     external_id (str, optional): the external id field for upsert operations. Defaults to 'External_Id__c'.
    #     contentType (str, optional): the content type of the file. Defaults to 'CSV', 'JSON' also accepted.
    # Returns: 
    #     response: a response object containg the job_id. For more information on the response object see https://www.w3schools.com/python/ref_requests_response.asp
    #     a response object see https://www.w3schools.com/python/ref_requests_response.asp
        
    # Salesforce API docs: https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/create_job.htm    
    print(f"Creating job for {sf_object} with operation {operation}") 
    url = "https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest"

    # https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/datafiles_prepare_csv.htm
    ## we can set columnDelimiter to `,^,|,;,<tab>, and the default <comma>
    # sets the object to Contact, the content type to CSV, and the operation to insert
    payload_d = {
        "object": sf_object,
        "contentType": "CSV",
        # set columnDelimiter to TAB instead of comma for ease of dealing with commas in address fields
        #https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/create_job.htm
        "columnDelimiter": "TAB", 
        "operation": operation
    }
    
    # as per https://developer.salesforce.com/docs/atlas.en-us.api_asynch.meta/api_asynch/walkthrough_upsert.htm
    if operation=='upsert':
        payload_d['externalIdFieldName']=external_id
    print(operation, payload_d)        
    payload = json.dumps(payload_d)
    
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)
    self.bulk_job_id = response.json()['id']
    return response 


In [ ]:
#| export
@patch
def upload_csv(self : Salesforce, 
                obj_s: str = "", # Salesforce object to upload 
                num_rows: int = 100, # the number of rows to upload 
                ) -> requests.Response:
    """Using the job_id from the previous step, upload the csv file to the job

    Args:
        file (filepointer): file pointer to the csv filek
    """
    # if not(file):
    #     # throw error
    #     assert False, "File not found"

    print(f"Uploading {num_rows if num_rows else 'all'} rows to job {self.bulk_job_id} of object {obj_s}")

    if len(obj_s)==0: # no object provided
        assert False, "obj_s must not be empty"
    # if not(self.atms):
    #     # throw error
    #     assert False, "Salesforce.atms must be assigned"
    
            
    # file_path_s = os.path.join(self.atms.download_dir , f"atms_transformed_{obj_s}.csv")
    file_path_s = os.path.join(Salesforce.class_download_dir , f"{obj_s}.csv")

    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/batches"

    # replace all occurrences of '\2019' with \'
    # we may have done this in ATMS already, but just in case
    try:
        for line in fileinput.input(files=file_path_s, inplace=True):
            line = line.replace('\u2019', "'")
            print(line, end='')

        # _df : pd.Dataframe = pd.read_csv(file_path_s, sep='\t')
        
        # if num_rows == 0:
        #     num_rows = len(_df)
        # else:
        #     num_rows = min(num_rows, len(_df))
        # payload : dict = _df[- num_rows:].to_dict()
        with open(file_path_s,'r') as payload:
            headers = {
            'Content-Type': 'text/csv',
            'Authorization': f'Bearer {self.sf_access_token}'
            }
            response = requests.request("PUT", url, headers=headers, data=payload)
    except FileNotFoundError:
        print("File not found error in Saleforce.upload_csv: ", file_path_s)
        return None
    
    return response
   

In [ ]:
#| export 
@patch
def close_job(self: Salesforce):
    # close the job (from Postman)
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}"

    payload = json.dumps({
        "state": "UploadComplete"
    })
    headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("PATCH", url, headers=headers, data=payload)

    print(response.text)
    return response.json()
     

In [ ]:
#| export       
# get job status (from Postman)
@patch
def job_status(self: Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}"

    payload = {}
    headers = {
    'Authorization': f'Bearer {self.sf_access_token}'
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()



In [ ]:
#| export
@patch
def successful_results(self : Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/successfulResults"

    payload = {}
    headers = {
        'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response


In [ ]:
#| export
@patch
def failed_results(self: Salesforce):
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/jobs/ingest/{self.bulk_job_id}/failedResults"

    payload = {}
    headers = {
        'Authorization': f'Bearer {self.sf_access_token}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    # 
    return response


In [ ]:
#| export
@patch
def get_sf_object_ids(self: Salesforce, 
                      object: str = 'Contact' # REST endpoint for data object
                      ):
    """Get Safesforce IDs for a the specified object

    """
    print(f"Retrieving Object Ids for {object} from Salesforce")
    sf_headers = { 'Authorization': f"Bearer {self.sf_access_token}", 'Content-Type': 'application/json' }
    end_point ="https://cremaconsulting-dev-ed.develop.my.salesforce.com"
    service = "/services/data/v57.0/"
    r = requests.request("GET", end_point+service+f"query/?q=SELECT+Id+FROM+{object}", headers=sf_headers, data={})
    assert isinstance(r.json(), dict), f"response: {r.json()}, header: {sf_headers}"
    object_ids = [d.get('Id') for d in r.json()['records']]
    while r.json()['done'] == False:
        new_url = end_point+r.json()['nextRecordsUrl']
        print(new_url)
        r = requests.request("GET", new_url, headers=sf_headers, data={})
        print((r.json()))
        fresh_object_ids = [d.get('Id') for d in r.json()['records']]
        print(len(fresh_object_ids))   
        object_ids+=fresh_object_ids
        
    print('total number of objects = ',len(object_ids))
    return object_ids


In [ ]:
#| export
@patch
def delete_sf_objects(self: Salesforce, 
                      obj_s: str = 'Contact'
                      ):
    """Delete Salesforce objects"""
    object_ids = self.get_sf_object_ids(obj_s)
    file_path_s = os.path.join(Salesforce.class_download_dir , f"{obj_s}.csv")
    print(f"In Salesforce.delete_sf_objects: Deleting {len(object_ids)} {obj_s} objects using {file_path_s}")
    with open(file_path_s, 'w') as f:
        f.write('Id\n')
        for id in object_ids:
            f.write(id+'\n')
    self.create_job( obj_s, 'delete')
    self.upload_csv(obj_s, num_rows=0) # upload all rows
    sleep(2)
    self.close_job()
    sleep(10)
    self.successful_results()
        


In [ ]:
#| export
@patch
def test_sf_object_load_and_delete(self: Salesforce, 
        sf_object_s : str = None, # Salesforce API endpoint
        input_file_s: str = None, # local file name
        remove_sf_objs: bool = False # remove the data just added to Salesforce
        ):
    """Test loading a Salesforce object with data from a local file"""
    assert sf_object_s
    assert input_file_s

    # sf.create_job('MembershipMembers__c', contentType='CSV')
    self.create_job(sf_object_s, contentType='CSV')
    print("Salesforce job id: ", self.bulk_job_id)

    #replace 
    # culprit is \u2019 - it cannot be encoded in latin-1 codec
    self.upload_csv(input_file_s)
    
        

    self.close_job()
    self.failed_results()
    self.successful_results()
    self.job_status()

    if remove_sf_objs:
        self.delete_sf_objects('membershipTerm__c')

In [ ]:
#| export


mem_s = "[].{membershipId__c: membershipId, \
    memberSince__c: memberSince, \
    updateDate__c: updateDate}"

memTerm_s = "[:50].membershipTerms[].{membershipTermId__c: membershipTermId,\
membershipKey__r_1_membershipId__c:membershipKey,\
effectiveDate__c:effectiveDate,\
expiryDate__c:expiryDate,\
membershipType__c:membershipType,\
upgradeFromTermKey__c:upgradeFromTermKey,\
giftMembership__c:giftMembership,\
refunded__c:refunded,\
saleDetailKey__c:saleDetailKey,\
itemKey__c:itemKey}"

memMembers_s = "[].membershipTerms[].membershipMembers[].{membershipMemberId__c:membershipMemberId,\
membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
cardNumber__c:cardNumber,\
membershipNumber__c:membershipNumber,\
cardStatus__c:cardStatus,\
contactKey__c:contactKey,\
displayName__c:displayName}"

@patch
def process_memberships(self: Salesforce ):
    """Unpack memberships data from atms object and write to membership, membership_terms, and membership_members csv files."""

    mem_d = { 'memberships': {'fname':'Membership__c.csv', 'jmespath': mem_s},
               'membership_terms': {'fname':'MembershipTerm.csv','jmespath': memTerm_s},
               'membership_members': {'fname': 'MembershipMember.csv', 'jmespath': memMembers_s}
                }
            

    if not ('memberships' in self.atms.obj_d):
        self.atms.load_data_file_to_dict('memberships')
        assert 'memberships' in self.atms.obj_d, f"memberships not in atms.obj_d {self.atms.obj_d.keys()}"
    
    atms_d = self.atms.obj_d['memberships']

    for key, v_pair in mem_d.items():
        file_path_s = os.path.join(Salesforce.class_download_dir, v_pair['fname'])
        dict_l = jp.search(v_pair['jmespath'], atms_d)
        with open(file_path_s, 'w') as f:
            # hack to create header with a dot in it, jmespath won't do it
            f.write('\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()]) + '\n') # header
            for d in dict_l:
                #changed this to not write None for empty values, eg "" for null and false (a default value)
                f.write('\t'.join([str(v) if v else "" for v in d.values()]) + '\n')
    

In [ ]:
#| export
search_s = "[].{LastName: organizationName,\
    MailingPostalCode: addresses[0].postalZipCode,\
    MailingCity: addresses[0].city,\
    MailingStreet: addresses[0].line1, \
    MailingCountry: addresses[0].country, \
    Phone: phones[?phoneType == 'Business'].phoneNumber | [0],\
    Email: emails[0].address[0],\
    External_Id__c: contactId}"

import re

def escape_quotes(text):
    # Escape single quotes
    # text = re.sub(r"\'", r"\\'", text)
    text = re.sub(r"\'", r"_", text)
    # Escape double quotes
    text = re.sub(r'\"', r'_', text)
    # text = re.sub(r',', r'*', text) ## shouldn't be necessary with tab delimiter
    # text = re.sub(r'\"', r'\\"', text)
    return text.strip()

@patch
def process_contacts(self: Salesforce ):
    """ unpack contacts data from atms object and write to contacts csv file."""
    if not ('contacts' in self.atms.obj_d):
        self.atms.load_data_file_to_dict('contacts')
        assert 'contacts' in self.atms.obj_d, f"contacts not in atms.obj_d {self.atms.obj_d.keys()}"
    
    file_path_s = os.path.join(Salesforce.class_download_dir, 'Contact.csv')
    dict_l = jp.search(search_s, self.atms.obj_d['contacts'])

    columnDelimiter = '\t'
    with open(file_path_s, 'w') as f:
        header = columnDelimiter.join(dict_l[0].keys())
        f.write(header+'\n')
        for item in dict_l:
            l = [escape_quotes(str(v)) if v else " " for v in item.values()]
            f.write(columnDelimiter.join(l)+'\n')

In [ ]:
sf = Salesforce()
sf._sf_access_token

Directory 'atms_download' already exists.


'00D8Y000001RMKv!AQwAQMO5wbpk2GWBzavd6yTB6IPfp7woqoTC45gd8SCjLXmQOku1WctiO3z3OQ7Z68_tmrYiAwk6PywIi70DfscCwCSyro.L'

In [ ]:
atms = ATMS_api()
sf.atms = atms
atms.retrieve_and_clean('memberships')

Directory 'atms_download' already exists.
my id is w3st4dl6
download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6
resp_d = self.get_telus_data(memberships,offset=0, count= 1000, since_date=)
resp_d = self.get_telus_data(memberships,offset=1000, count= 1000, since_date=)
cleaning_data_file - download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6
creating file:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_memberships.json
Finished cleaning atms_memberships.json -> /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_memberships.json
ATMS_api - Attempting to load:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_memberships.json  into dict


In [ ]:
#| export
a = 3

In [ ]:
atms.retrieve_and_clean('contacts', max_rows=5 )
sf.process_contacts()
display( pd.read_csv('sf_download/contacts.csv', sep='\t') )
sf.create_job(sf_object='Contact', operation='insert')


download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6
resp_d = self.get_telus_data(contacts,offset=0, count= 5, since_date=)
cleaning_data_file - download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6
creating file:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_contacts.json
Finished cleaning atms_contacts.json -> /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_contacts.json
ATMS_api - Attempting to load:  /Users/josephmann/Documents/Github/doubledot/atms_download/w3st4dl6/atms_transformed_contacts.json  into dict


,LastName,MailingPostalCode,MailingCity,MailingStreet,MailingCountry,Phone,Email,External_Id__c
0,J E Lapointe School,T4X 1K2,Beaumont,4801 55 Avenue,Canada,7809295988,pam.jenkins@blackgold.ca,83
1,New Humble Centre School,T0C 0V0,Calmar,PO Box 780,,7809853211,,84
2,New Sarepta Community High School,T0B 3M0,New Sarepta,PO Box 10,Canada,7809413924,,85
3,New Sarepta Elementary School,T0B 3M0,New Sarepta,5051-2 Street S,Canada,7809413927,tobey.morris@gshare.blackgold.ca,86
4,Thorsby Elementary School,T0C 2P0,Thorsby,PO Box 240,Canada,7807893776,Raylene.jubinville@blackgold.ca,87


Creating job for Contact with operation insert
insert {'object': 'Contact', 'contentType': 'CSV', 'columnDelimiter': 'TAB', 'operation': 'insert'}
{"id":"7508Y00000mVFuhQAG","operation":"insert","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-10T06:28:28.000+0000","systemModstamp":"2023-05-10T06:28:28.000+0000","state":"Open","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0,"contentUrl":"services/data/v57.0/jobs/ingest/7508Y00000mVFuhQAG/batches","lineEnding":"LF","columnDelimiter":"TAB"}


<Response [200]>

In [ ]:
!cat sf_download/contacts.csv

LastName	MailingPostalCode	MailingCity	MailingStreet	MailingCountry	Phone	Email	External_Id__c
J E Lapointe School	T4X 1K2	Beaumont	4801 55 Avenue	Canada	7809295988	pam.jenkins@blackgold.ca	83
New Humble Centre School	T0C 0V0	Calmar	PO Box 780	 	7809853211	 	84
New Sarepta Community High School	T0B 3M0	New Sarepta	PO Box 10	Canada	7809413924	 	85
New Sarepta Elementary School	T0B 3M0	New Sarepta	5051-2 Street S	Canada	7809413927	tobey.morris@gshare.blackgold.ca	86
Thorsby Elementary School	T0C 2P0	Thorsby	PO Box 240	Canada	7807893776	Raylene.jubinville@blackgold.ca	87


In [ ]:
sf.upload_csv('Contact')

Uploading 100 rows to job 7508Y00000mVFuhQAG of object Contact


<Response [201]>

In [ ]:
sf.close_job()

{"id":"7508Y00000mVFuhQAG","operation":"insert","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-10T06:28:28.000+0000","systemModstamp":"2023-05-10T06:28:28.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}


{'id': '7508Y00000mVFuhQAG',
 'operation': 'insert',
 'object': 'Contact',
 'createdById': '0058Y00000CIEn9QAH',
 'createdDate': '2023-05-10T06:28:28.000+0000',
 'systemModstamp': '2023-05-10T06:28:28.000+0000',
 'state': 'UploadComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 57.0}

In [ ]:
sf.job_status()

{'id': '7508Y00000mVFuhQAG',
 'operation': 'insert',
 'object': 'Contact',
 'createdById': '0058Y00000CIEn9QAH',
 'createdDate': '2023-05-10T06:28:28.000+0000',
 'systemModstamp': '2023-05-10T06:28:34.000+0000',
 'state': 'UploadComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 57.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'TAB',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [ ]:
r = sf.successful_results()
print(r)
print(r.text)

<Response [200]>
"sf__Id"	"sf__Created"	LastName	MailingPostalCode	MailingCity	MailingStreet	MailingCountry	Phone	Email	External_Id__c
"0038Y00003eRh5YQAS"	"true"	"New Humble Centre School"	"T0C 0V0"	"Calmar"	"PO Box 780"	" "	"7809853211"	" "	"84"
"0038Y00003eRh5ZQAS"	"true"	"New Sarepta Community High School"	"T0B 3M0"	"New Sarepta"	"PO Box 10"	"Canada"	"7809413924"	" "	"85"



In [ ]:
r = sf.failed_results()
print(r)
print(r.text)
# print(r.json())

<Response [200]>
"sf__Id"	"sf__Error"	LastName	MailingPostalCode	MailingCity	MailingStreet	MailingCountry	Phone	Email	External_Id__c
""	"DUPLICATES_DETECTED:Use one of these records?:--"	"J E Lapointe School"	"T4X 1K2"	"Beaumont"	"4801 55 Avenue"	"Canada"	"7809295988"	"pam.jenkins@blackgold.ca"	"83"
""	"DUPLICATES_DETECTED:Use one of these records?:--"	"New Sarepta Elementary School"	"T0B 3M0"	"New Sarepta"	"5051-2 Street S"	"Canada"	"7809413927"	"tobey.morris@gshare.blackgold.ca"	"86"
""	"DUPLICATES_DETECTED:Use one of these records?:--"	"Thorsby Elementary School"	"T0C 2P0"	"Thorsby"	"PO Box 240"	"Canada"	"7807893776"	"Raylene.jubinville@blackgold.ca"	"87"



JSONDecodeError: Extra data: line 1 column 10 (char 9)

In [ ]:
sf.get_sf_object_ids('Contact')

Retrieving Object Ids for Contact from Salesforce
total number of objects =  24


['0038Y00003bx8WPQAY',
 '0038Y00003bx8WQQAY',
 '0038Y00003bx8WRQAY',
 '0038Y00003bx8WUQAY',
 '0038Y00003bx8WVQAY',
 '0038Y00003bx8WWQAY',
 '0038Y00003bx8WXQAY',
 '0038Y00003bx8WYQAY',
 '0038Y00003bx8WZQAY',
 '0038Y00003bx8WaQAI',
 '0038Y00003bx8WcQAI',
 '0038Y00003bx8WdQAI',
 '0038Y00003bx8WeQAI',
 '0038Y00003bx8WfQAI',
 '0038Y00003bx8WgQAI',
 '0038Y00003eRh5MQAS',
 '0038Y00003eRh5NQAS',
 '0038Y00003eRh5OQAS',
 '0038Y00003eRh5PQAS',
 '0038Y00003eRh5QQAS',
 '0038Y00003eRh5TQAS',
 '0038Y00003eRh5UQAS',
 '0038Y00003eRh5YQAS',
 '0038Y00003eRh5ZQAS']

In [ ]:
sf.delete_sf_objects('Contact')

Retrieving Object Ids for Contact from Salesforce
total number of objects =  24
In Salesforce.delete_sf_objects: Deleting 24 Contact objects using /Users/josephmann/Documents/Github/doubledot/sf_download/Contact.csv
Creating job for Contact with operation delete
delete {'object': 'Contact', 'contentType': 'CSV', 'columnDelimiter': 'TAB', 'operation': 'delete'}
{"id":"7508Y00000mVFzIQAW","operation":"delete","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-10T06:36:17.000+0000","systemModstamp":"2023-05-10T06:36:17.000+0000","state":"Open","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0,"contentUrl":"services/data/v57.0/jobs/ingest/7508Y00000mVFzIQAW/batches","lineEnding":"LF","columnDelimiter":"TAB"}
Uploading all rows to job 7508Y00000mVFzIQAW of object Contact
{"id":"7508Y00000mVFzIQAW","operation":"delete","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-10T06:36:17.000+0000","systemModstamp":"2023-05-10T06:3

In [ ]:
sf.get_sf_object_ids('Contact')

Retrieving Object Ids for Contact from Salesforce
total number of objects =  15


['0038Y00003bx8WPQAY',
 '0038Y00003bx8WQQAY',
 '0038Y00003bx8WRQAY',
 '0038Y00003bx8WUQAY',
 '0038Y00003bx8WVQAY',
 '0038Y00003bx8WWQAY',
 '0038Y00003bx8WXQAY',
 '0038Y00003bx8WYQAY',
 '0038Y00003bx8WZQAY',
 '0038Y00003bx8WaQAI',
 '0038Y00003bx8WcQAI',
 '0038Y00003bx8WdQAI',
 '0038Y00003bx8WeQAI',
 '0038Y00003bx8WfQAI',
 '0038Y00003bx8WgQAI']

In [ ]:
r = sf.successful_results()
print(r.text)



"sf__Id"	"sf__Created"	Id
"0038Y00003eRh5MQAS"	"false"	"0038Y00003eRh5MQAS"
"0038Y00003eRh5NQAS"	"false"	"0038Y00003eRh5NQAS"
"0038Y00003eRh5OQAS"	"false"	"0038Y00003eRh5OQAS"
"0038Y00003eRh5PQAS"	"false"	"0038Y00003eRh5PQAS"
"0038Y00003eRh5QQAS"	"false"	"0038Y00003eRh5QQAS"
"0038Y00003eRh5TQAS"	"false"	"0038Y00003eRh5TQAS"
"0038Y00003eRh5UQAS"	"false"	"0038Y00003eRh5UQAS"
"0038Y00003eRh5YQAS"	"false"	"0038Y00003eRh5YQAS"
"0038Y00003eRh5ZQAS"	"false"	"0038Y00003eRh5ZQAS"



JSONDecodeError: Extra data: line 1 column 10 (char 9)

In [ ]:
sf.get_sf_object_ids('Membership__c')

Retrieving Object Ids for Membership__c from Salesforce
total number of objects =  0


[]

In [ ]:
sf.get_sf_object_ids('MembershipTerm__c')

Retrieving Object Ids for MembershipTerm__c from Salesforce
total number of objects =  0


[]

In [ ]:
sf.delete_sf_objects('Membership__c')

Retrieving Object Ids for Membership__c from Salesforce
https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/query/0r88Y1h0EONu6wyQYB-2000
{'totalSize': 3000, 'done': True, 'records': [{'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/sobjects/Membership__c/a028Y00001Eu0mnQAB'}, 'Id': 'a028Y00001Eu0mnQAB'}, {'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/sobjects/Membership__c/a028Y00001Eu0moQAB'}, 'Id': 'a028Y00001Eu0moQAB'}, {'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/sobjects/Membership__c/a028Y00001Eu0mpQAB'}, 'Id': 'a028Y00001Eu0mpQAB'}, {'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/sobjects/Membership__c/a028Y00001Eu0mqQAB'}, 'Id': 'a028Y00001Eu0mqQAB'}, {'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/sobjects/Membership__c/a028Y00001Eu0mrQAB'}, 'Id': 'a028Y00001Eu0mrQAB'}, {'attributes': {'type': 'Membership__c', 'url': '/services/data/v57.0/s

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
sf.successful_results()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()